In [48]:
# Preparación y Limpieza de Datos - Modelo de Fuga Colsubsidio
# ==============================================================
# 
# Objetivo: Limpiar e integrar los datasets para crear una base sólida
# - Limpieza de variables financieras con formato monetario
# - Manejo inteligente de valores faltantes por tipo de variable
# - Integración robusta de múltiples fuentes de datos
# - Validación de calidad y consistencia post-limpieza
# - Preparación de datos listos para feature engineering

# =============================================================================
# CARGA DE DATOS
# =============================================================================
import sys
import pandas as pd
from pathlib import Path

# Ruta base de los datos
data_path = Path("../data/raw")

# Cargar datasets principales
train = pd.read_excel(data_path / "train.xlsx")
test = pd.read_excel(data_path / "test.xlsx")

# Inicializar diccionario para guardar todos los datasets cargados
datasets = {
    "train": train,
    "test": test
}

#  Cargar datos complementarios
try:
    demograficas = pd.read_excel(data_path / "train_test_demograficas.xlsx")
    subsidios = pd.read_excel(data_path / "train_test_subsidios.xlsx")
    
    datasets["demograficas"] = demograficas
    datasets["subsidios"] = subsidios
    
    # Integrar datos en train y test
    train_integrated = train.merge(demograficas, on="id", how="left") \
                                       .merge(subsidios, on="id", how="left")
    test_integrated = test.merge(demograficas, on="id", how="left") \
                                     .merge(subsidios, on="id", how="left")
    
    datasets["train_integrated"] = train_integrated
    datasets["test_integrated"] = test_integrated
    
except FileNotFoundError:
    print("⚠️ Archivos complementarios no encontrados. Usando solo datos principales.")

# Imprimir información de todos los datasets cargados
print("\n=== DATASETS CARGADOS ===")
for name, df in datasets.items():
    if isinstance(df, pd.DataFrame):
        print(f"{name.upper()}: {len(df):,} registros x {len(df.columns)} columnas")


=== DATASETS CARGADOS ===
TRAIN: 50,001 registros x 22 columnas
TEST: 5,001 registros x 20 columnas
DEMOGRAFICAS: 55,002 registros x 10 columnas
SUBSIDIOS: 55,002 registros x 4 columnas
TRAIN_INTEGRATED: 50,001 registros x 34 columnas
TEST_INTEGRATED: 5,001 registros x 32 columnas


In [49]:
# =============================================================================
# LIMPIEZA VARIABLES FINANCIERAS TRAIN
# =============================================================================

def limpiar_financiera(serie, nombre):
   """Convierte formato '1 050 000.00' a float"""
   serie_clean = serie.astype(str).str.replace(' ', '').str.replace(',', '')
   serie_clean = serie_clean.replace('nan', np.nan)
   return pd.to_numeric(serie_clean, errors='coerce')

print(f"\nLimpieza variables financieras:")

train_clean = train.copy()

# Variables que requieren limpieza de formato
vars_monetarias = [
   'Saldo', 'Limite.Cupo', 'Disponible.Avances', 'Limite.Avances',
   'Total.Intereses', 'Saldos.Mes.Ant', 'Pagos.Mes.Ant', 'Vtas.Mes.Ant',
   'Pago.del.Mes', 'Pago.Minimo', 'Vr.Mora', 'Vr.Cuota.Manejo'
]

for var in vars_monetarias:
   if var in train_clean.columns:
       antes = train_clean[var].dtype
       train_clean[var] = limpiar_financiera(train_clean[var], var)
       despues = train_clean[var].dtype
       valores_validos = train_clean[var].notna().sum()
       print(f"  {var}: {antes} → {despues} ({valores_validos:,} válidos)")

# Verificar limpieza exitosa
vars_texto_restantes = train_clean.select_dtypes(include=['object']).columns
vars_financieras_texto = [v for v in vars_texto_restantes if v in vars_monetarias]

if not vars_financieras_texto:
   print(f"  ✓ Limpieza completada - todas las variables financieras son numéricas")
else:
   print(f"  ⚠ Variables pendientes: {vars_financieras_texto}")

# Estadísticas básicas post-limpieza
print(f"\nEstadísticas principales:")
vars_principales = ['Saldo', 'Limite.Cupo', 'Edad.Mora', 'Vr.Mora']

for var in vars_principales:
   if var in train_clean.columns:
       stats = train_clean[var].describe()
       ceros = (train_clean[var] == 0).sum()
       print(f"  {var}:")
       print(f"    Rango: ${stats['min']:,.0f} - ${stats['max']:,.0f}")
       print(f"    Promedio: ${stats['mean']:,.0f}")
       print(f"    Ceros: {ceros:,} ({ceros/len(train_clean)*100:.1f}%)")

datasets["train_clean"] = train_clean


Limpieza variables financieras:
  Saldo: object → float64 (50,001 válidos)
  Limite.Cupo: object → float64 (50,001 válidos)
  Disponible.Avances: object → float64 (50,001 válidos)
  Limite.Avances: object → float64 (50,001 válidos)
  Total.Intereses: object → float64 (50,001 válidos)
  Saldos.Mes.Ant: object → float64 (50,001 válidos)
  Pagos.Mes.Ant: object → float64 (50,001 válidos)
  Vtas.Mes.Ant: object → float64 (50,001 válidos)
  Pago.del.Mes: object → float64 (50,001 válidos)
  Pago.Minimo: object → float64 (50,001 válidos)
  Vr.Mora: object → float64 (50,001 válidos)
  Vr.Cuota.Manejo: object → float64 (50,001 válidos)
  ✓ Limpieza completada - todas las variables financieras son numéricas

Estadísticas principales:
  Saldo:
    Rango: $0 - $18,937,684
    Promedio: $340,878
    Ceros: 24,047 (48.1%)
  Limite.Cupo:
    Rango: $150,000 - $1,400,000,000
    Promedio: $1,265,604
    Ceros: 0 (0.0%)
  Edad.Mora:
    Rango: $0 - $4,050
    Promedio: $90
    Ceros: 41,316 (82.6%)
  

In [50]:
# =============================================================================
# LIMPIEZA VARIABLES FINANCIERAS TEST
# =============================================================================
print(f"\nLimpiando test...")

test_clean = test.copy()

for var in vars_monetarias:
    if var in test_clean.columns:
        test_clean[var] = limpiar_financiera(test_clean[var], var)

datasets["test_clean"] = test_clean

# %%
# Integrar datos limpios con demograficas y subsidios
train_full = train_clean.merge(demograficas, on='id', how='left').merge(subsidios, on='id', how='left')
test_full = test_clean.merge(demograficas, on='id', how='left').merge(subsidios, on='id', how='left')

print(f"Train integrado: {train_full.shape}")
print(f"Test integrado: {test_full.shape}")

# %%
# Manejar faltantes
train_full = train_full.fillna(0)
test_full = test_full.fillna(0)

# %%
# Verificar y guardar
print(f"Target ratio: {train_full['Target'].value_counts()[0] / train_full['Target'].value_counts()[1]:.0f}:1")

train_full.to_csv('data/processed/train_prepared.csv', index=False)
test_full.to_csv('data/processed/test_prepared.csv', index=False)

print("Datos preparados - listos para feature engineering")


Limpiando test...
Train integrado: (50001, 34)
Test integrado: (5001, 32)
Target ratio: 34:1
Datos preparados - listos para feature engineering


In [51]:
# =============================================================================
# VALIDACION FINAL
# =============================================================================
print("=== VALIDACIÓN FINAL ===")

# 1. Verificar dimensiones
print(f"Dimensiones:")
print(f"  Train: {train_full.shape}")
print(f"  Test: {test_full.shape}")

# 2. Verificar variables financieras son numéricas
financial_check = []
for var in vars_monetarias:
   if var in train_full.columns:
       is_numeric = train_full[var].dtype in ['int64', 'float64']
       financial_check.append(f"  {var}: {'✓' if is_numeric else '✗'} {train_full[var].dtype}")

print(f"\nVariables financieras:")
for check in financial_check:
   print(check)

# 3. Verificar target
target_dist = train_full['Target'].value_counts()
print(f"\nTarget:")
print(f"  No Fuga: {target_dist[0]:,}")
print(f"  Fuga: {target_dist[1]:,}")
print(f"  Ratio: {target_dist[0]/target_dist[1]:.0f}:1")

# 4. Verificar faltantes
print(f"\nFaltantes:")
print(f"  Train: {train_full.isnull().sum().sum()}")
print(f"  Test: {test_full.isnull().sum().sum()}")

# 5. Verificar consistencia columnas (sin target)
train_cols = set(train_full.columns) - {'Target'}
test_cols = set(test_full.columns)
consistent = train_cols == test_cols

print(f"\nConsistencia columnas: {'✓' if consistent else '✗'}")
if not consistent:
   print(f"  Solo en train: {train_cols - test_cols}")
   print(f"  Solo en test: {test_cols - train_cols}")

print(f"\n{' VALIDACIÓN EXITOSA' if consistent and train_full.isnull().sum().sum() == 0 else '⚠️ REVISAR ISSUES'}")

=== VALIDACIÓN FINAL ===
Dimensiones:
  Train: (50001, 34)
  Test: (5001, 32)

Variables financieras:
  Saldo: ✓ float64
  Limite.Cupo: ✓ float64
  Disponible.Avances: ✓ float64
  Limite.Avances: ✓ float64
  Total.Intereses: ✓ float64
  Saldos.Mes.Ant: ✓ float64
  Pagos.Mes.Ant: ✓ float64
  Vtas.Mes.Ant: ✓ float64
  Pago.del.Mes: ✓ float64
  Pago.Minimo: ✓ float64
  Vr.Mora: ✓ float64
  Vr.Cuota.Manejo: ✓ float64

Target:
  No Fuga: 48,589
  Fuga: 1,412
  Ratio: 34:1

Faltantes:
  Train: 0
  Test: 0

Consistencia columnas: ✗
  Solo en train: {'Retencion'}
  Solo en test: set()

⚠️ REVISAR ISSUES


In [52]:
# =============================================================================
# ELIMINAR VARIABLES INNECESARIAS
# =============================================================================
train_full = train_full.drop(['Retencion'], axis=1)

print(f"Train después: {train_full.shape}")
print(f"Test: {test_full.shape}")

# Verificar consistencia
train_cols = set(train_full.columns) - {'Target'}
test_cols = set(test_full.columns)
print(f"Consistencia: {'✓' if train_cols == test_cols else '✗'}")

# %%
# Guardar versión final
train_full.to_csv('data/processed/train_prepared.csv', index=False)
test_full.to_csv('data/processed/test_prepared.csv', index=False)

print(" PREPARACIÓN COMPLETADA")
print(f"Train final: {train_full.shape}")
print(f"Test final: {test_full.shape}")


Train después: (50001, 33)
Test: (5001, 32)
Consistencia: ✓
 PREPARACIÓN COMPLETADA
Train final: (50001, 33)
Test final: (5001, 32)


In [53]:
# =============================================================================
# CORRECCIÓN FECHAS SEGÚN DICCIONARIO
# =============================================================================
print("=== CORRECCIÓN FECHAS (según diccionario) ===")

fecha_cols = ['Fecha.Expedicion', 'Fecha.Proceso']

for col in fecha_cols:
    if col in train_full.columns:
        print(f"\n{col}:")
        print(f"  Tipo actual: {train_full[col].dtype}")
        print(f"  Muestra antes: {train_full[col].head(2).tolist()}")
        
        # Conversión según diccionario
        train_full[col] = pd.to_datetime(train_full[col], errors='coerce', dayfirst=True)
        test_full[col] = pd.to_datetime(test_full[col], errors='coerce', dayfirst=True)
        
        print(f"  Tipo después: {train_full[col].dtype}")
        print(f"  Muestra después: {train_full[col].head(2).tolist()}")
        print(f"  Nulos: {train_full[col].isnull().sum()}")

# %%
# Guardar con fechas corregidas
train_full.to_csv('data/processed/train_prepared.csv', index=False)
test_full.to_csv('data/processed/test_prepared.csv', index=False)



=== CORRECCIÓN FECHAS (según diccionario) ===

Fecha.Expedicion:
  Tipo actual: object
  Muestra antes: [datetime.datetime(2006, 4, 11, 0, 0), '17/08/2007']
  Tipo después: datetime64[ns]
  Muestra después: [Timestamp('2006-04-11 00:00:00'), Timestamp('2007-08-17 00:00:00')]
  Nulos: 0

Fecha.Proceso:
  Tipo actual: object
  Muestra antes: [datetime.datetime(2018, 1, 4, 0, 0), datetime.datetime(2018, 1, 4, 0, 0)]
  Tipo después: datetime64[ns]
  Muestra después: [Timestamp('2018-01-04 00:00:00'), Timestamp('2018-01-04 00:00:00')]
  Nulos: 0


In [54]:
# =============================================================================
# TRATAMIENTO DE OUTLIERS
# =============================================================================

print("=== TRATAMIENTO DE OUTLIERS ===")

def tratar_outliers_financieros(df):
   df_clean = df.copy()
   
   # Variables financieras principales
   vars_financieras = ['Saldo', 'Limite.Cupo', 'Pago.del.Mes', 'Vr.Mora', 'Disponible.Avances']
   
   for var in vars_financieras:
       if var in df_clean.columns:
           # Calcular percentiles
           p99 = df_clean[var].quantile(0.99)
           outliers_antes = (df_clean[var] > p99).sum()
           
           # Aplicar cap en P99
           df_clean[var] = np.where(df_clean[var] > p99, p99, df_clean[var])
           
           print(f"  {var}: {outliers_antes} outliers tratados (cap en ${p99:,.0f})")
   
   return df_clean

# Aplicar a ambos datasets
print("Tratando outliers en train...")
train_full = tratar_outliers_financieros(train_full)

print("\nTratando outliers en test...")
test_full = tratar_outliers_financieros(test_full)

print("\n Outliers tratados - datos más robustos para modelado")

# %%
# RE-GUARDAR DATOS CON OUTLIERS TRATADOS
train_full.to_csv('data/processed/train_prepared.csv', index=False)
test_full.to_csv('data/processed/test_prepared.csv', index=False)

print(" Datos re-guardados con outliers tratados")
print("Archivos actualizados para feature engineering")

=== TRATAMIENTO DE OUTLIERS ===
Tratando outliers en train...
  Saldo: 500 outliers tratados (cap en $3,118,276)
  Limite.Cupo: 499 outliers tratados (cap en $8,950,000)
  Pago.del.Mes: 500 outliers tratados (cap en $281,700)
  Vr.Mora: 500 outliers tratados (cap en $650,521)
  Disponible.Avances: 500 outliers tratados (cap en $4,066,641)

Tratando outliers en test...
  Saldo: 50 outliers tratados (cap en $2,895,680)
  Limite.Cupo: 50 outliers tratados (cap en $8,300,000)
  Pago.del.Mes: 50 outliers tratados (cap en $200,000)
  Vr.Mora: 50 outliers tratados (cap en $576,696)
  Disponible.Avances: 50 outliers tratados (cap en $4,000,000)

 Outliers tratados - datos más robustos para modelado
 Datos re-guardados con outliers tratados
Archivos actualizados para feature engineering


In [55]:
# =============================================================================
# GUARDAR DATOS PREPARADOS
# =============================================================================
train_full.to_csv('data/processed/train_prepared.csv', index=False)
test_full.to_csv('data/processed/test_prepared.csv', index=False)
